# Query ESGF input4MIPs index and extract source_id entries

## notes

## imports

In [1]:
import json
import numpy as np
import os
import pdb
import requests
from IPython.display import clear_output

## example source_id queries

## get source_id entries

In [2]:
# both input4MIPs and input4mips
project = "input4MIPs"

# Dataset search
inputsD = "https://esgf-node.llnl.gov/solr/datasets/select?q=*:*&rows=0&wt=json&facet=true&" \
         "fq=type:Dataset&fq=replica:false&fq=project:" + project + "&facet.field=source_id"
js = requests.get(inputsD)
js_mipsD = json.loads(js.text)
print(inputsD)

# File search
inputsF = "https://esgf-node.llnl.gov/solr/files/select?q=*:*&rows=0&wt=json&facet=true&" \
          "fq=type:File&fq=replica:false&fq=project:" + project + "&facet.field=source_id"
js = requests.get(inputsF)
js_mipsF = json.loads(js.text)
print(inputsF)

https://esgf-node.llnl.gov/solr/datasets/select?q=*:*&rows=0&wt=json&facet=true&fq=type:Dataset&fq=replica:false&fq=project:input4MIPs&facet.field=source_id
https://esgf-node.llnl.gov/solr/files/select?q=*:*&rows=0&wt=json&facet=true&fq=type:File&fq=replica:false&fq=project:input4MIPs&facet.field=source_id


## extract source_id entries and composite

In [3]:
dicInpmD = js_mipsD["facet_counts"]["facet_fields"]["source_id"]
print("len(dicInpmD):", len(dicInpmD))
dicInpmF = js_mipsF["facet_counts"]["facet_fields"]["source_id"]
print("len(dicInpmF):", len(dicInpmF))
#print(dicInpmD)
#print(dicInpmF)

len(dicInpmD): 368
len(dicInpmF): 346


In [4]:
# datasets
srcIdLen = len(dicInpmD)
print("srcIdDLen:", srcIdLen)
els = np.arange(0, srcIdLen, 2)
srcIdDDict = {}
srcIds = dicInpmD  # reset to generic variable name
counts = 0
for cnt, srcId in enumerate(els.tolist()):
    #print(cnt, srcId)
    srcIdDDict[srcIds[srcId]] = srcIds[srcId+1]
    counts += int(srcIds[srcId+1])

print("len(srcIdDDict.keys()):", len(srcIdDDict.keys()))
print("dataset counts:", counts)
# sort dictionary
#sorted(srcIdDDict.items())
#print(srcIdDDict)
srcIdDDictList = list(sorted(srcIdDDict.keys()))
#print(srcIdDDictList)

# files
srcIdLen = len(dicInpmF)
print("srcIdFLen:", srcIdLen)
els = np.arange(0, srcIdLen, 2)
srcIdFDict = {}
srcIds = dicInpmF  # reset to generic variable name
counts = 0
for cnt, srcId in enumerate(els.tolist()):
    #print(cnt, srcId)
    srcIdFDict[srcIds[srcId]] = srcIds[srcId+1]
    counts += int(srcIds[srcId+1])

print("len(srcIdFDict.keys()):", len(srcIdFDict.keys()))
print("dataset counts:", counts)
# sort dictionary
srcIdFDictList = list(sorted(srcIdFDict.keys()))
#print(srcIdFDictList)

# determine missing
print("Search results: Dataset includes, excluded from File searches:")
set(srcIdDDictList).difference(srcIdFDictList)

srcIdDLen: 368
len(srcIdDDict.keys()): 184
dataset counts: 5876
srcIdFLen: 346
len(srcIdFDict.keys()): 173
dataset counts: 9373
Search results: Dataset includes, excluded from File searches:


{'CCMI-hist-nat-1-0',
 'CCMI-hist-sol-1-0',
 'CCMI-hist-volc-1-0',
 'IAMC-AIM-ssp370-1-0',
 'IAMC-GCAM4-ssp434-1-0',
 'IAMC-GCAM4-ssp460-1-0',
 'IAMC-IMAGE-ssp119-1-0',
 'IAMC-IMAGE-ssp126-1-0',
 'IAMC-MESSAGE-GLOBIOM-ssp245-1-0',
 'IAMC-REMIND-MAGPIE-ssp534-over-1-0',
 'IAMC-REMIND-MAGPIE-ssp585-1-0'}

## example source_id queries

## using source_id entries from Dataset search build a library

In [6]:
solrQry = "https://esgf-node.llnl.gov/esg-search/search/?limit=1000&format=application%2Fsolr%2Bjson&source_id=" \
          "PLACEHOLDER" + "&project=input4mips&project=input4MIPs&distrib=false&fields=*"  # all fields
mstrJson = {}  # create catch dictionary
oF = "tmp.json"
for count, srcId in enumerate(srcIdDDict.keys()):
    print(count, srcId)
    qryStr = solrQry.replace("PLACEHOLDER", srcId)
    #print("qryStr:", qryStr)
    js = requests.get(qryStr)
    js_srcId = json.loads(js.text)
    #print(js_srcId["response"]["docs"][0])
    # write to placeholder to test
    srcIdLen = len(js_srcId["response"]["docs"])
    #print("\033[H\033[xJ", end="")  # This should clean screen
    # https://stackoverflow.com/questions/517970/how-to-clear-the-interpreter-console
    clear_output(wait=False)
    # https://stackoverflow.com/questions/24816237/ipython-notebook-clear-cell-output-in-code
    print("len(js_srcId):", srcIdLen)
    for entry in np.arange(0, srcIdLen):
        a = js_srcId["response"]["docs"][entry]
        instId = a["instance_id"]
        mstrJson[instId] = a        
        if os.path.exists(oF):
            os.remove(oF)
        fH = open(oF, "w")
        json.dump(a, fH, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":"),)
        #json.dump(mstrJson, fH, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":"),)
        fH.close()
    #pdb.set_trace()
print("All done")
# Write all out
oF = "comp.json"
if os.path.exists(oF):
    os.remove(oF)
fH = open(oF, "w")
json.dump(mstrJson, fH, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":"),)
fH.close()

len(js_srcId): 247
All done
